In [12]:
import torch
from models.cgan import GeneratorModel, DiscriminatorModel
import numpy as np
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from metric.niqe import niqe
from metric.fid import calculate_fid
from skimage.transform import rescale, resize
from imageio import imread
from collections import defaultdict

In [3]:
g = GeneratorModel()
d = DiscriminatorModel()
gsd, dsd = torch.load('cgan.pt', map_location='cpu')
g.load_state_dict(gsd)
d.load_state_dict(dsd)

<All keys matched successfully>

In [4]:
transform_fn = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(178), transforms.Resize(64)])
train_data = datasets.CelebA('./data', split='train', download=True, transform=transform_fn)

Files already downloaded and verified


In [5]:
img, label = train_data[0]

In [6]:
g = g.cuda()
d = d.cuda()
img  = img.cuda()
label = label.cuda()

In [17]:
niqe_dic = defaultdict(list)
for index in range(10):
    img, label = train_data[index]
    img  = img.cuda()
    label = label.cuda()
    f, axs = plt.subplots(5, label.size(0) // 5, figsize=(16, 10))
    axs = axs.flatten()
    o = np.copy(img.cpu().numpy())
    o.resize(200, 200)
    niqe_dic['ori'].append(niqe(np.copy(o)))
    for i in range(label.size(0)):
        label[i] = 1 - label[i]
        with torch.no_grad():
            o = g(img.unsqueeze(0), torch.randint(0, 1, label.size()).unsqueeze(0).cuda())
        o = np.moveaxis(o.squeeze().cpu().numpy(), 0, 2)
        label[i] = 1 - label[i]
        axs[i].imshow(o)
        axs[i].axis('off')
        o = np.copy(np.mean(o, axis=2))
        o.resize(200, 200)
        niqe_dic[i].append(niqe(np.copy(o)))
    plt.savefig(f'samples/cgan/{index:5d}.png')
    plt.close()